In [1]:
import pandas as pd
import pyxlsb

In [ ]:
estoque_dtype = {
    'Código Produto':int,
    'Código Estabelecimento':int,
    'Total': 'int64',
    'Data':object
}


estoque = pd.read_excel("Estoque_Por_Dia.xlsx",
usecols=['Código Produto','Código Estabelecimento','Total','Data'],
parse_dates=['Data'],
dtype=estoque_dtype)

In [ ]:
estoque.head(10)

,Código Produto,Ano,Nome Mês,dia,Código Estabelecimento,Total,Data,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,t
0,4110020335,2025,Julho,8,804,1.0,2025-07-08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4110020335,2025,Julho,8,806,1.0,2025-07-08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4110020335,2025,Julho,9,804,1.0,2025-07-09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4110020335,2025,Julho,9,806,1.0,2025-07-09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4110020335,2025,Julho,10,804,1.0,2025-07-10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4110020335,2025,Julho,10,806,1.0,2025-07-10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4110020335,2025,Julho,11,804,1.0,2025-07-11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4110020335,2025,Julho,11,806,1.0,2025-07-11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4110020335,2025,Julho,12,804,1.0,2025-07-12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4110020335,2025,Julho,12,806,1.0,2025-07-12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
